## Understanding problem statement

- Objective: Create a ML model that optimizes the channel distribution by maximizing market reach and revenue

### Phase 1: EDA
- Understand how variables differ across channels and region
- Visualize on week, city and channel levels
- Insights: Action items to optmize distribution channel

## Ingesting the data

In [ ]:
# READING ALL THE DATA IN PANDAS DATAFRAME

import pandas as pd
city_df = pd.read_csv('../../data/raw/city.csv', encoding='ISO-8859-1')
competitors_df = pd.read_csv('../../data/raw/competitivelandscape.csv')
consumer_behaviour_df = pd.read_csv('../../data/raw/consumerbehavior.csv')
market_influencers_df = pd.read_csv('../../data/raw/externalmarketinfluencers.csv')
products_df = pd.read_csv('../../data/raw/products.csv')
sales_df = pd.read_csv('../../data/raw/retailsalesdistribution.csv')

## Univariate 

### Sales data

In [ ]:
sales_df.head()

In [ ]:
from ydata_profiling import ProfileReport

sales_data_profile = ProfileReport(sales_df, title='Sales Data Profiling Report', explorative=True)


In [ ]:
sales_data_profile.to_file('../../eda/sales_data_profile.html')

- 20 cities, 7 SKUs, 5 channels

### Competitive Behavior

In [ ]:
from ydata_profiling import ProfileReport

competitors_data_profile = ProfileReport(competitors_df, title='competitors_Data Profiling Report', explorative=True)

competitors_data_profile.to_file('../../eda/competitors_data_profile.html')


### Products 

In [ ]:
from ydata_profiling import ProfileReport

products_data_profile = ProfileReport(products_df, title='Products Data Profiling Report', explorative=True)

products_data_profile.to_file('../../eda/products_data_profile.html')


### External Market Influence

In [ ]:
from ydata_profiling import ProfileReport

external_market_influence_data_profile = ProfileReport(market_influencers_df, title='External Market Influence Data Profiling Report', explorative=True)

external_market_influence_data_profile.to_file('../../eda/external_market_influence_data_profile.html')


### Customer Behavior 

In [ ]:
from ydata_profiling import ProfileReport

customer_behavior_data_profile = ProfileReport(consumer_behaviour_df, title='Customer Behavior Data Profiling Report', explorative=True)

customer_behavior_data_profile.to_file('../../eda/customer_behavior_data_profile.html')


### City

In [ ]:
from ydata_profiling import ProfileReport

city_data_profile = ProfileReport(city_df, title='City Data Profiling Report', explorative=True)

city_data_profile.to_file('../../eda/city_data_profile.html')


# Data Preprocessing

### Sales Data

In [ ]:

# STANDARDIZING THE DATE FORMAT IN SALES DATAFRAME
import pandas as pd
sales_df_processed = sales_df.copy()

sales_df_processed['Date'] = pd.to_datetime(sales_df_processed['Date'], format='%Y-%m-%d')

sales_df_processed['Date'] = pd.to_datetime(sales_df_processed['Date']).dt.strftime('%Y-%m-%d')

sales_df_processed.head()



In [ ]:
# save data
sales_df_processed.to_csv('../../data/processed/sales_processed.csv', index=False)

### Competitor Data

In [ ]:

# STANDARDIZING THE DATE FORMAT IN SALES DATAFRAME
import pandas as pd
competitors_df_processed = competitors_df.copy()

competitors_df_processed['Date'] = pd.to_datetime(competitors_df_processed['Date'], format='%Y-%m-%d')

competitors_df_processed['Date'] = pd.to_datetime(competitors_df_processed['Date']).dt.strftime('%Y-%m-%d')

competitors_df_processed.head()



In [ ]:
# save data
competitors_df_processed.to_csv('../../data/processed/competitors_processed.csv', index=False)

### Products

In [ ]:
import pandas as pd
import re
import numpy as np

products_df_processed = products_df.copy()

# Convert "Flavor Variant" column to categorical data type
products_df_processed['Flavor Variant'] = products_df_processed['Flavor Variant'].astype('category')


# 2. Convert Launch Date from MM/DD/YYYY to yyyy-mm-dd
products_df_processed['Launch Date'] = pd.to_datetime(products_df_processed['Launch Date (MM/DD/YYYY)'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

def convert_pack_size(pack_str):
    """
    Extract numeric value and convert pack size to litres.
    Assumes the pack size string contains a number and a unit (ml or L).
    """
    match = re.search(r'([\d\.]+)\s*(ml|l)', pack_str, re.IGNORECASE)
    if match:
        value = float(match.group(1))
        unit = match.group(2).lower()
        if unit == 'ml':
            return value / 1000  # convert millilitres to litres
        else:
            return value
    return np.nan

products_df_processed['Pack Size (L)'] = products_df_processed['Pack Size (ml/L)'].apply(convert_pack_size)

channels = {
    'General Trade': r'general trade',
    'E Commerce': r'e[-\s]?commerce',
    'Modern Trade': r'modern trade',
    'HoReCa': r'horeca',
    'Q Commerce': r'q[-\s]?commerce'
}

def encode_distribution(dist_str):
    """
    Returns a Series with one-hot encoding for each distribution channel.
    """
    dist_lower = dist_str.lower()
    result = {}
    for channel, pattern in channels.items():
        result[channel] = 1 if re.search(pattern, dist_lower) else 0
    return pd.Series(result)

dist_df = products_df_processed['Distribution Coverage'].apply(encode_distribution)
products_df_processed = pd.concat([products_df_processed, dist_df], axis=1)  

products_df_processed = products_df_processed[['Product Name', 'Flavor Variant', 'SKU Identification Number', 
          'Launch Date', 'Pack Size (L)', 'General Trade', 'E Commerce', 
          'Modern Trade', 'HoReCa', 'Q Commerce']]

products_df_processed.head(100)






In [ ]:
# save data
products_df_processed.to_csv('../../data/processed/products_processed.csv', index=False)

### External Market influence

In [ ]:
import pandas as pd

market_influencers_df_processed = market_influencers_df.copy()

# convert data to yyyy-mm-dd format
market_influencers_df_processed['Week_Start_Date'] = pd.to_datetime(market_influencers_df_processed['Week_Start_Date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')

market_influencers_df_processed['Festival'] = market_influencers_df_processed['Festival'].fillna("No Festival")


market_influencers_df_processed.head()


In [ ]:
# save data
market_influencers_df_processed.to_csv('../../data/processed/market_influencers_processed.csv', index=False)

### Customer Behavior

In [ ]:

import pandas as pd

consumer_behaviour_df_processed = consumer_behaviour_df.copy()
consumer_behaviour_df_processed['Purchase_Frequency'] = consumer_behaviour_df_processed['Purchase_Frequency'].fillna(consumer_behaviour_df_processed['Purchase_Frequency'].median())

consumer_behaviour_df_processed['Preferred_Channel'] = consumer_behaviour_df_processed['Preferred_Channel'].replace('HoReca', 'HoReCa')

consumer_behaviour_df_processed['Active'] = consumer_behaviour_df_processed['Active'].map({True: 1, False: 0})



consumer_behaviour_df_processed.head()


In [ ]:
# save data
consumer_behaviour_df_processed.to_csv('../../data/processed/consumer_behavior_processed.csv', index=False)

### City

In [ ]:

import pandas as pd

city_df_processed = city_df.copy()

city_df_processed = city_df_processed.dropna(axis=1, how='all')
city_df_processed['City_Name'] = city_df_processed['City_Name'].astype('category')

city_df_processed['Per_Capita_Income (INR)'] = city_df_processed['Per_Capita_Income (INR)'].astype(str).str.replace(',', '')
city_df_processed['Per_Capita_Income (INR)'] = pd.to_numeric(city_df_processed['Per_Capita_Income (INR)'], errors='coerce')

city_df_processed.head()



In [ ]:
# save data
city_df_processed.to_csv('../../data/processed/city_processed.csv', index=False)

## Generating reports of processed data

In [ ]:
# READ CITY, COMPETITOR, CONSUMER BEHAVIOR, MARKET INFLUENCE, PRODUCTS AND SALES DATA FROM PROCESSED FOLDER AND CREATE Y DATA PROFILING REPORTS AND SAVE THEM IN EDA/PROCESSED FOLDER
import pandas as pd
from ydata_profiling import ProfileReport

city_df = pd.read_csv('../../data/processed/city_processed.csv')
competitors_df = pd.read_csv('../../data/processed/competitors_processed.csv')
consumer_behaviour_df = pd.read_csv('../../data/processed/consumer_behavior_processed.csv')
market_influencers_df = pd.read_csv('../../data/processed/market_influencers_processed.csv')
products_df = pd.read_csv('../../data/processed/products_processed.csv')
sales_df = pd.read_csv('../../data/processed/sales_processed.csv')


sales_data_profile = ProfileReport(sales_df, title='Sales Data Profiling Report', explorative=True)

sales_data_profile.to_file('../../eda/reports/processed/sales_data_profile.html')

competitors_data_profile = ProfileReport(competitors_df, title='competitors_Data Profiling Report', explorative=True)

competitors_data_profile.to_file('../../eda/reports/processed/competitors_data_profile.html')

products_data_profile = ProfileReport(products_df, title='Products Data Profiling Report', explorative=True)

products_data_profile.to_file('../../eda/reports/processed/products_data_profile.html')

external_market_influence_data_profile = ProfileReport(market_influencers_df, title='External Market Influence Data Profiling Report', explorative=True)

external_market_influence_data_profile.to_file('../../eda/reports/processed/external_market_influence_data_profile.html')

customer_behavior_data_profile = ProfileReport(consumer_behaviour_df, title='Customer Behavior Data Profiling Report', explorative=True)

customer_behavior_data_profile.to_file('../../eda/reports/processed/customer_behavior_data_profile.html')

city_data_profile = ProfileReport(city_df, title='City Data Profiling Report', explorative=True)

city_data_profile.to_file('../../eda/reports/processed/city_data_profile.html')



# EDA

In [1]:
import pandas as pd
sales_df = pd.read_csv('../../data/processed/sales_processed.csv')
sales_df.head()

,Date,City_ID,SKU_ID,Channel,Units_Sold,Sales
0,2023-01-01,CT001,SKU1002,Q Commerce,268,80.4
1,2023-01-01,CT001,SKU1004,Q Commerce,168,50.4
2,2023-01-01,CT001,SKU1001,E Commerce,521,156.3
3,2023-01-01,CT001,SKU1002,E Commerce,247,74.1
4,2023-01-01,CT001,SKU1007,E Commerce,161,12075.0


In [11]:
import plotly.express as px

# Sales by SKU (Revenue)
sales_sku = sales_df.groupby('SKU_ID', as_index=False)['Sales'].sum()
fig = px.bar(sales_sku, x='SKU_ID', y='Sales', title='Sales Revenue by SKU')
fig.show()

# Sales by Channel
sales_channel = sales_df.groupby('Channel', as_index=False)['Sales'].sum()
fig = px.pie(sales_channel, names='Channel', values='Sales', title='Sales Distribution by Channel')
fig.show()

# Sales by Channel
sales_channel = sales_df.groupby('Channel', as_index=False)['Units_Sold'].sum()
fig = px.pie(sales_channel, names='Channel', values='Units_Sold', title='Units_Sold Distribution by Channel')
fig.show()


# Sales by City
# Sales by City
sales_city = sales_df.groupby('City_ID', as_index=False)['Sales'].sum()
fig = px.pie(sales_city, names='City_ID', values='Sales', title='Sales Revenue by City')
fig.show()

# Sales by Date
sales_date = sales_df.groupby('Date', as_index=False).agg({'Sales': 'sum', 'Units_Sold': 'sum'})
fig = px.line(sales_date, x='Date', y=['Sales', 'Units_Sold'], title='Sales Revenue and Units Sold by Date')
fig.show()


In [1]:
import pandas as pd
import plotly.express as px

# Load datasets
city = pd.read_csv('../../data/processed/city_processed.csv')
competitor = pd.read_csv('../../data/processed/competitors_processed.csv')
consumer = pd.read_csv('../../data/processed/consumer_behavior_processed.csv')
external = pd.read_csv('../../data/processed/market_influencers_processed.csv')
sales = pd.read_csv('../../data/processed/sales_processed.csv')
products = pd.read_csv('../../data/processed/products_processed.csv')

# Create Selling_Price = Sales / Units_Sold
sales['Selling_Price'] = sales['Sales'] / sales['Units_Sold']

# Convert dates to datetime in all datasets
sales['Date'] = pd.to_datetime(sales['Date'])
external['Week_Start_Date'] = pd.to_datetime(external['Week_Start_Date'])
competitor['Date'] = pd.to_datetime(competitor['Date'])



# Merge with city data
merged = sales.merge(city, on='City_ID', how='left')

merged = merged.merge(
    products, 
    left_on='SKU_ID', 
    right_on='SKU Identification Number', 
    how='left'
)

# merged = merged.merge(
#     consumer, 
#     on="City_ID",
#     how='left'
# )



# # Add week start date to sales data
# merged['Week_Start_Date'] = merged['Date'] - pd.to_timedelta(merged['Date'].dt.dayofweek, unit='D')

# # Merge with external market data (now aligned on datetime)
# merged = merged.merge(
#     external, 
#     left_on=['Week_Start_Date', 'City_ID'], 
#     right_on=['Week_Start_Date', 'City_ID'], 
#     how='left'
# )

# # Merge with competitor data (filter for Minute Maid)
# competitor_mm = competitor[competitor['Brand'] == 'Minute Maid']
# merged = merged.merge(
#     competitor_mm, 
#     left_on=['Week_Start_Date', 'Channel'], 
#     right_on=['Date', 'Channel'], 
#     suffixes=('', '_competitor'), 
#     how='left'
# )

# # Merge consumer behavior (aggregate by city and channel)
# # Standardize channel names to match (e.g., "E-commerce" vs "E Commerce")
# consumer['Preferred_Channel'] = consumer['Preferred_Channel'].str.replace('-', ' ', regex=False)
# merged['Channel'] = merged['Channel'].str.replace('-', ' ', regex=False)

# consumer_agg = consumer.groupby(['City_ID', 'Preferred_Channel']).agg(
#     Avg_Income=('Income_Level', 'mean'),
#     Preferred_Flavor=('Preferred_Flavor', lambda x: x.mode()[0]),
#     Active_Customers=('Active', 'sum')
# ).reset_index()

# merged = merged.merge(
#     consumer_agg, 
#     left_on=['City_ID', 'Channel'], 
#     right_on=['City_ID', 'Preferred_Channel'], 
#     how='left'
# )

merged.head(1000)

,Date,City_ID,SKU_ID,Channel,Units_Sold,Sales,Selling_Price,City_Name,City_tier,Population_Density(persons/km),...,Product Name,Flavor Variant,SKU Identification Number,Launch Date,Pack Size (L),General Trade,E Commerce,Modern Trade,HoReCa,Q Commerce
0,2023-01-01,CT001,SKU1002,Q Commerce,268,80.4,0.3,Delhi,Tier 1,14893,...,Minute Maid Mixed Fruit Juice,Mixed Fruit,SKU1002,2015-04-23,1.00,1,1,1,1,1
1,2023-01-01,CT001,SKU1004,Q Commerce,168,50.4,0.3,Delhi,Tier 1,14893,...,Minute Maid Pulpy Orange,Orange,SKU1004,2015-04-23,1.00,1,1,1,1,1
2,2023-01-01,CT001,SKU1001,E Commerce,521,156.3,0.3,Delhi,Tier 1,14893,...,Minute Maid Apple Juice - Honey Infused,Apple,SKU1001,2022-02-19,1.00,1,1,0,0,0
3,2023-01-01,CT001,SKU1002,E Commerce,247,74.1,0.3,Delhi,Tier 1,14893,...,Minute Maid Mixed Fruit Juice,Mixed Fruit,SKU1002,2015-04-23,1.00,1,1,1,1,1
4,2023-01-01,CT001,SKU1007,E Commerce,161,12075.0,75.0,Delhi,Tier 1,14893,...,Minute Maid 250ml Pulpy Orange,Orange,SKU1007,2015-04-23,0.25,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-01-03,CT008,SKU1007,Modern Trade,803,60225.0,75.0,Pune,Tier 1,6034,...,Minute Maid 250ml Pulpy Orange,Orange,SKU1007,2015-04-23,0.25,1,1,1,1,0
996,2023-01-03,CT008,SKU1004,Modern Trade,209,62.7,0.3,Pune,Tier 1,6034,...,Minute Maid Pulpy Orange,Orange,SKU1004,2015-04-23,1.00,1,1,1,1,1
997,2023-01-03,CT008,SKU1008,Modern Trade,167,12525.0,75.0,Pune,Tier 1,6034,...,Minute Maid 250ml Mixed Fruit Juice,Mixed Fruit,SKU1008,2015-04-23,0.25,1,1,1,1,0
998,2023-01-03,CT008,SKU1001,General Trade,2366,709.8,0.3,Pune,Tier 1,6034,...,Minute Maid Apple Juice - Honey Infused,Apple,SKU1001,2022-02-19,1.00,1,1,0,0,0


In [20]:
merged.to_csv('../../data/aggregated/merged_data_daily_granularity.csv', index=False)

In [2]:
# drop City_ID, SKU_ID, Product_Name, SKU Identification Name, Launch Date
merged = pd.read_csv('../../data/aggregated/merged_data_daily_granularity.csv')
# merged = merged.drop(columns=['City_ID', 'SKU_ID', 'Product Name', 'SKU Identification Number', 'Launch Date', "Channel", "Selling_Price"])
merged.head(1000)

,Date,City_ID,SKU_ID,Channel,Units_Sold,Sales,Selling_Price,City_Name,City_tier,Population_Density(persons/km),...,Product Name,Flavor Variant,SKU Identification Number,Launch Date,Pack Size (L),General Trade,E Commerce,Modern Trade,HoReCa,Q Commerce
0,2023-01-01,CT001,SKU1002,Q Commerce,268,80.4,0.3,Delhi,Tier 1,14893,...,Minute Maid Mixed Fruit Juice,Mixed Fruit,SKU1002,2015-04-23,1.00,1,1,1,1,1
1,2023-01-01,CT001,SKU1004,Q Commerce,168,50.4,0.3,Delhi,Tier 1,14893,...,Minute Maid Pulpy Orange,Orange,SKU1004,2015-04-23,1.00,1,1,1,1,1
2,2023-01-01,CT001,SKU1001,E Commerce,521,156.3,0.3,Delhi,Tier 1,14893,...,Minute Maid Apple Juice - Honey Infused,Apple,SKU1001,2022-02-19,1.00,1,1,0,0,0
3,2023-01-01,CT001,SKU1002,E Commerce,247,74.1,0.3,Delhi,Tier 1,14893,...,Minute Maid Mixed Fruit Juice,Mixed Fruit,SKU1002,2015-04-23,1.00,1,1,1,1,1
4,2023-01-01,CT001,SKU1007,E Commerce,161,12075.0,75.0,Delhi,Tier 1,14893,...,Minute Maid 250ml Pulpy Orange,Orange,SKU1007,2015-04-23,0.25,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-01-03,CT008,SKU1007,Modern Trade,803,60225.0,75.0,Pune,Tier 1,6034,...,Minute Maid 250ml Pulpy Orange,Orange,SKU1007,2015-04-23,0.25,1,1,1,1,0
996,2023-01-03,CT008,SKU1004,Modern Trade,209,62.7,0.3,Pune,Tier 1,6034,...,Minute Maid Pulpy Orange,Orange,SKU1004,2015-04-23,1.00,1,1,1,1,1
997,2023-01-03,CT008,SKU1008,Modern Trade,167,12525.0,75.0,Pune,Tier 1,6034,...,Minute Maid 250ml Mixed Fruit Juice,Mixed Fruit,SKU1008,2015-04-23,0.25,1,1,1,1,0
998,2023-01-03,CT008,SKU1001,General Trade,2366,709.8,0.3,Pune,Tier 1,6034,...,Minute Maid Apple Juice - Honey Infused,Apple,SKU1001,2022-02-19,1.00,1,1,0,0,0


In [3]:
pd.set_option('display.max_columns', None)

# One-hot encode categorical columns
merged = pd.get_dummies(merged, columns=['City_Name', 'City_tier', 'Flavor Variant', 'Pack Size (L)'], dtype=int)
merged.head(1000)

,Date,City_ID,SKU_ID,Channel,Units_Sold,Sales,Selling_Price,Population_Density(persons/km),Per_Capita_Income (INR),Product Name,SKU Identification Number,Launch Date,General Trade,E Commerce,Modern Trade,HoReCa,Q Commerce,City_Name_Ahmedabad,City_Name_Bengaluru,City_Name_Bhopal,City_Name_Chandigarh,City_Name_Chennai,City_Name_Delhi,City_Name_Guwahati,City_Name_Hyderabad,City_Name_Jaipur,City_Name_Jammu,City_Name_Kochi,City_Name_Kolkata,City_Name_Lucknow,City_Name_Ludhiana,City_Name_Mumbai,City_Name_Nagpur,City_Name_Patna,City_Name_Pune,City_Name_Surat,City_Name_Visakhapatnam,City_tier_Tier 1,City_tier_Tier 2,Flavor Variant_Apple,Flavor Variant_Guava,Flavor Variant_Mixed Fruit,Flavor Variant_Orange,Pack Size (L)_0.135,Pack Size (L)_0.25,Pack Size (L)_1.0
0,2023-01-01,CT001,SKU1002,Q Commerce,268,80.4,0.3,14893,461910,Minute Maid Mixed Fruit Juice,SKU1002,2015-04-23,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
1,2023-01-01,CT001,SKU1004,Q Commerce,168,50.4,0.3,14893,461910,Minute Maid Pulpy Orange,SKU1004,2015-04-23,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1
2,2023-01-01,CT001,SKU1001,E Commerce,521,156.3,0.3,14893,461910,Minute Maid Apple Juice - Honey Infused,SKU1001,2022-02-19,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1
3,2023-01-01,CT001,SKU1002,E Commerce,247,74.1,0.3,14893,461910,Minute Maid Mixed Fruit Juice,SKU1002,2015-04-23,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
4,2023-01-01,CT001,SKU1007,E Commerce,161,12075.0,75.0,14893,461910,Minute Maid 250ml Pulpy Orange,SKU1007,2015-04-23,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-01-03,CT008,SKU1007,Modern Trade,803,60225.0,75.0,6034,336503,Minute Maid 250ml Pulpy Orange,SKU1007,2015-04-23,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0
996,2023-01-03,CT008,SKU1004,Modern Trade,209,62.7,0.3,6034,336503,Minute Maid Pulpy Orange,SKU1004,2015-04-23,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1
997,2023-01-03,CT008,SKU1008,Modern Trade,167,12525.0,75.0,6034,336503,Minute Maid 250ml Mixed Fruit Juice,SKU1008,2015-04-23,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0
998,2023-01-03,CT008,SKU1001,General Trade,2366,709.8,0.3,6034,336503,Minute Maid Apple Juice - Honey Infused,SKU1001,2022-02-19,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1


In [4]:
merged.to_csv('../../data/aggregated/merged_data_daily_granularity_ohe.csv', index=False)

In [15]:
import pandas as pd

df = pd.read_csv('../../data/aggregated/merged_data_daily_granularity_ohe.csv')

# convert Date to datetime object of format yyyy-mm-dd
df['Date'] = pd.to_datetime(df['Date'])

df.dtypes

Date                              datetime64[ns]
City_ID                                   object
SKU_ID                                    object
Channel                                   object
Units_Sold                                 int64
Sales                                    float64
Selling_Price                            float64
Population_Density(persons/km)             int64
Per_Capita_Income (INR)                    int64
Product Name                              object
SKU Identification Number                 object
Launch Date                               object
General Trade                              int64
E Commerce                                 int64
Modern Trade                               int64
HoReCa                                     int64
Q Commerce                                 int64
City_Name_Ahmedabad                        int64
City_Name_Bengaluru                        int64
City_Name_Bhopal                           int64
City_Name_Chandigarh

In [16]:

# Fetch the above columns
df = df[
    [
        'Date', 
        'Units_Sold', 
        'Sales', 
        'Selling_Price', 
        'Population_Density(persons/km)', 
        'Per_Capita_Income (INR)', 
        'City_Name_Ahmedabad', 
        'City_Name_Bengaluru', 
        'City_Name_Bhopal', 
        'City_Name_Chandigarh', 
        'City_Name_Chennai', 
        'City_Name_Delhi', 
        'City_Name_Guwahati', 
        'City_Name_Hyderabad', 
        'City_Name_Jaipur', 
        'City_Name_Jammu', 
        'City_Name_Kochi', 
        'City_Name_Kolkata', 
        'City_Name_Lucknow', 
        'City_Name_Ludhiana', 
        'City_Name_Mumbai', 
        'City_Name_Nagpur', 
        'City_Name_Patna', 
        'City_Name_Pune', 
        'City_Name_Surat', 
        'City_Name_Visakhapatnam', 
        'City_tier_Tier 1', 
        'City_tier_Tier 2', 
        'Flavor Variant_Apple', 
        'Flavor Variant_Guava', 
        'Flavor Variant_Mixed Fruit', 
        'Flavor Variant_Orange', 
        'Pack Size (L)_0.135', 
        'Pack Size (L)_0.25', 
        'Pack Size (L)_1.0'
    ]
]

df_grouped = df.groupby('Date').agg({
    'Units_Sold': 'sum',
    'Sales': 'sum',
    'Selling_Price': 'mean',
    'Population_Density(persons/km)': 'mean',
    'Per_Capita_Income (INR)': 'mean',
    'City_Name_Ahmedabad': 'sum',
    'City_Name_Bengaluru': 'sum',
    'City_Name_Bhopal': 'sum',
    'City_Name_Chandigarh': 'sum',
    'City_Name_Chennai': 'sum',
    'City_Name_Delhi': 'sum',
    'City_Name_Guwahati': 'sum',
    'City_Name_Hyderabad': 'sum',
    'City_Name_Jaipur': 'sum',
    'City_Name_Jammu': 'sum',
    'City_Name_Kochi': 'sum',
    'City_Name_Kolkata': 'sum',
    'City_Name_Lucknow': 'sum',
    'City_Name_Ludhiana': 'sum',
    'City_Name_Mumbai': 'sum',
    'City_Name_Nagpur': 'sum',
    'City_Name_Patna': 'sum',
    'City_Name_Pune': 'sum',
    'City_Name_Surat': 'sum',
    'City_Name_Visakhapatnam': 'sum',
    'City_tier_Tier 1': 'sum',
    'City_tier_Tier 2': 'sum',
    'Flavor Variant_Apple': 'sum',
    'Flavor Variant_Guava': 'sum',
    'Flavor Variant_Mixed Fruit': 'sum',
    'Flavor Variant_Orange': 'sum',
    'Pack Size (L)_0.135': 'sum',
    'Pack Size (L)_0.25': 'sum',
    'Pack Size (L)_1.0': 'sum'
}).reset_index()

df_grouped.head(1000)

,Date,Units_Sold,Sales,Selling_Price,Population_Density(persons/km),Per_Capita_Income (INR),City_Name_Ahmedabad,City_Name_Bengaluru,City_Name_Bhopal,City_Name_Chandigarh,City_Name_Chennai,City_Name_Delhi,City_Name_Guwahati,City_Name_Hyderabad,City_Name_Jaipur,City_Name_Jammu,City_Name_Kochi,City_Name_Kolkata,City_Name_Lucknow,City_Name_Ludhiana,City_Name_Mumbai,City_Name_Nagpur,City_Name_Patna,City_Name_Pune,City_Name_Surat,City_Name_Visakhapatnam,City_tier_Tier 1,City_tier_Tier 2,Flavor Variant_Apple,Flavor Variant_Guava,Flavor Variant_Mixed Fruit,Flavor Variant_Orange,Pack Size (L)_0.135,Pack Size (L)_0.25,Pack Size (L)_1.0
0,2023-01-01,133098,2881566.0,30.671429,8730.2,313292.1,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,168,252,40,0,200,180,20,160,240
1,2023-01-02,131995,2780910.0,30.671429,8730.2,313292.1,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,168,252,40,0,200,180,20,160,240
2,2023-01-03,132096,2848796.7,30.671429,8730.2,313292.1,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,168,252,40,0,200,180,20,160,240
3,2023-01-04,132530,2819088.9,30.671429,8730.2,313292.1,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,168,252,40,0,200,180,20,160,240
4,2023-01-05,133672,2769576.6,30.671429,8730.2,313292.1,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,168,252,40,0,200,180,20,160,240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2024-12-27,165415,2970576.9,28.030435,8730.2,313292.1,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,184,276,40,40,200,180,20,160,280
727,2024-12-28,164602,3091994.4,28.030435,8730.2,313292.1,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,184,276,40,40,200,180,20,160,280
728,2024-12-29,166162,3165291.0,28.030435,8730.2,313292.1,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,184,276,40,40,200,180,20,160,280
729,2024-12-30,166345,3057104.4,28.030435,8730.2,313292.1,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,184,276,40,40,200,180,20,160,280


In [17]:
df_grouped.to_csv('../../data/aggregated/merged_grouped_sales_data.csv', index=False)

In [35]:
# read competitors data
import pandas as pd

df = pd.read_csv('../../data/processed/competitors_processed.csv')
df['Date'] = pd.to_datetime(df['Date'])

df = pd.get_dummies(df, columns=['Channel', 'Brand'], dtype=int)

df = df.groupby('Date').agg({
    'Mentions_Count': 'sum',
    'Sentiment_Score': 'mean',
    # 'Share_of_Voice': 'sum',  
    'Channel_E Commerce': 'sum',
    'Channel_General Trade': 'sum',
    'Channel_HoReCa': 'sum',
    'Channel_Modern Trade': 'sum',
    'Channel_Q Commerce': 'sum',
    'Brand_Amazon Solimo': 'sum',
    'Brand_B Natural': 'sum',
    'Brand_Minute Maid': 'sum',
    'Brand_Paper Boat': 'sum',
    'Brand_Real Fruit Juice': 'sum',
    'Brand_Tropicana': 'sum'
}).reset_index()

print(df.shape)

df.head(1000)

(105, 14)


,Date,Mentions_Count,Sentiment_Score,Channel_E Commerce,Channel_General Trade,Channel_HoReCa,Channel_Modern Trade,Channel_Q Commerce,Brand_Amazon Solimo,Brand_B Natural,Brand_Minute Maid,Brand_Paper Boat,Brand_Real Fruit Juice,Brand_Tropicana
0,2023-01-02,4027,69.116667,3,2,3,2,2,1,2,5,2,1,1
1,2023-01-09,4899,68.725000,3,2,3,2,2,1,2,5,2,1,1
2,2023-01-16,6050,69.975000,3,2,3,2,2,1,2,5,2,1,1
3,2023-01-23,4087,71.891667,3,2,3,2,2,1,2,5,2,1,1
4,2023-01-30,4732,67.008333,3,2,3,2,2,1,2,5,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,2024-12-02,4887,67.733333,3,2,3,2,2,1,2,5,2,1,1
101,2024-12-09,4716,71.616667,3,2,3,2,2,1,2,5,2,1,1
102,2024-12-16,5095,68.241667,3,2,3,2,2,1,2,5,2,1,1
103,2024-12-23,4371,69.375000,3,2,3,2,2,1,2,5,2,1,1


In [22]:
df.to_csv('../../data/aggregated/competitors_weekly_data.csv', index=False)

In [34]:

import pandas as pd

# read market data
df = pd.read_csv('../../data/processed/market_influencers_processed.csv')
city_df = pd.read_csv('../../data/processed/city_processed.csv')

# convert Week_Start_Date to datetime object of format yyyy-mm-dd
df['Week_Start_Date'] = pd.to_datetime(df['Week_Start_Date'])

df = pd.get_dummies(df, columns=['Festival', 'Weather_Type'], dtype=int)
# group by Week_Start_Date and aggregate
df = df.groupby('Week_Start_Date').agg({
    'Avg_Temperature': 'mean',
    'Festival_Diwali': 'max',
    'Festival_Holi': 'max',
    'Festival_No Festival': 'max',
    'Festival_Pongal': 'max',
    'Weather_Type_Cold': 'max',
    'Weather_Type_Hot': 'max',
    'Weather_Type_Mild': 'max',
    'Weather_Type_Rainy': 'max'
}).reset_index()

print(df.shape)
df.head()

(105, 10)


,Week_Start_Date,Avg_Temperature,Festival_Diwali,Festival_Holi,Festival_No Festival,Festival_Pongal,Weather_Type_Cold,Weather_Type_Hot,Weather_Type_Mild,Weather_Type_Rainy
0,2023-01-02,20.105,0,0,1,0,1,0,1,0
1,2023-01-09,20.440,0,0,0,1,1,0,1,0
2,2023-01-16,20.305,0,0,1,0,1,0,1,0
3,2023-01-23,20.825,0,0,1,0,1,0,1,0
4,2023-01-30,21.845,0,0,1,0,1,0,1,0


In [30]:
df.to_csv('../../data/aggregated/market_influencers_weekly_data.csv', index=False)

In [37]:
import pandas as pd 

market = pd.read_csv('../../data/aggregated/market_influencers_weekly_data.csv')
competitors = pd.read_csv('../../data/aggregated/competitors_weekly_data.csv')
print(market.shape)
print(competitors.shape)

# merge on Date and Week_Start_Date
df = market.merge(competitors, left_on='Week_Start_Date', right_on='Date', how='left')
df = df.drop(columns=['Week_Start_Date'])
print(df.shape)
df.head(1000)

(105, 10)
(105, 14)
(105, 23)


,Avg_Temperature,Festival_Diwali,Festival_Holi,Festival_No Festival,Festival_Pongal,Weather_Type_Cold,Weather_Type_Hot,Weather_Type_Mild,Weather_Type_Rainy,Date,Mentions_Count,Sentiment_Score,Channel_E Commerce,Channel_General Trade,Channel_HoReCa,Channel_Modern Trade,Channel_Q Commerce,Brand_Amazon Solimo,Brand_B Natural,Brand_Minute Maid,Brand_Paper Boat,Brand_Real Fruit Juice,Brand_Tropicana
0,20.105,0,0,1,0,1,0,1,0,2023-01-02,4027,69.116667,3,2,3,2,2,1,2,5,2,1,1
1,20.440,0,0,0,1,1,0,1,0,2023-01-09,4899,68.725000,3,2,3,2,2,1,2,5,2,1,1
2,20.305,0,0,1,0,1,0,1,0,2023-01-16,6050,69.975000,3,2,3,2,2,1,2,5,2,1,1
3,20.825,0,0,1,0,1,0,1,0,2023-01-23,4087,71.891667,3,2,3,2,2,1,2,5,2,1,1
4,21.845,0,0,1,0,1,0,1,0,2023-01-30,4732,67.008333,3,2,3,2,2,1,2,5,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,21.060,0,0,1,0,1,0,1,0,2024-12-02,4887,67.733333,3,2,3,2,2,1,2,5,2,1,1
101,20.510,0,0,1,0,1,0,0,0,2024-12-09,4716,71.616667,3,2,3,2,2,1,2,5,2,1,1
102,20.565,0,0,1,0,1,0,1,0,2024-12-16,5095,68.241667,3,2,3,2,2,1,2,5,2,1,1
103,19.880,0,0,1,0,1,0,0,0,2024-12-23,4371,69.375000,3,2,3,2,2,1,2,5,2,1,1


In [38]:
df.to_csv('../../data/aggregated/market_competitors_influence_weekly_data.csv', index=False)